# Парсинг вакансий с LinkedIn

## Описание задачи

Распарсить предоставленный csv файл с помощью BS 4, создав следующие признаки:

- наименование вакансии
- город
- страна
- тип занятости (online, hybride, on-site)
- компания
- размер компании (количество работников)
- сфера деятельности компании
- требуемые хард скилы
- дата публикации вакансии
- количество кандидатов на вакансию

## Часть1. Парсинг нужных данных

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import seaborn as sns
from datetime import timedelta, datetime
import requests
from geopy.geocoders import Nominatim
from googletrans import Translator
pd.options.mode.chained_assignment = None

In [2]:
data = pd.read_csv('/Users/denis_sl/Desktop/проект linkedin/masterskaya_yandex_2022_09_07.csv')
data.head(10)

,Unnamed: 0,html
0,0,"\n <div>\n <div class=""\n jobs-deta..."
1,1,"\n <div>\n <div class=""\n jobs-deta..."
2,2,"\n <div>\n <div class=""\n jobs-deta..."
3,3,"\n <div>\n <div class=""\n jobs-deta..."
4,4,"\n <div>\n <div class=""\n jobs-deta..."
5,5,"\n <div>\n <div class=""\n jobs-deta..."
6,6,"\n <div>\n <div class=""\n jobs-deta..."
7,7,"\n <div>\n <div class=""\n jobs-deta..."
8,8,"\n <div>\n <div class=""\n jobs-deta..."
9,9,"\n <div>\n <div class=""\n jobs-deta..."


### Название позиции

In [3]:
data['title'] = data['html'].apply(lambda x: BeautifulSoup(x).find('h2').text.strip())

### Количество кандидатов

In [4]:
def get_candidats(cell):
    try:
        return BeautifulSoup(cell).find('span', {'class': 'jobs-unified-top-card__applicant-count'}).text.strip().replace('applicants', '').replace('applicant','')
    except:
        return np.nan
        

In [5]:
data['applicants'] = data['html'].apply(get_candidats)
data['applicants'] = data['applicants'].astype('Int64')

### Ссылка

In [6]:
data['link'] = data['html'].apply(lambda x: "https://linkedin.com" + BeautifulSoup(x).find('a').get('href'))

### Геолокоция с разбивкой на город и страну

In [7]:
def get_geo(cell):
    try:
        return BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__bullet').text.strip()
    except:
        return np.nan

In [8]:
data['location'] = data['html'].apply(get_geo)

In [9]:
def get_city(cell):
    if len(cell.split(',')) > 1:
        return cell.split(',')[0].strip()
    elif "Metropolitan" in cell or "Greater" in cell:
        return cell.replace('Greater', '').replace('Metropolitan', '').replace('Area', '').replace('Region','').strip()
    else:
        return np.nan

In [10]:
data['city'] = data['location'].apply(get_city)

In [11]:
def get_country(cell):
    try:
        if len(cell.split(',')) > 1:
            return cell.split(',')[-1].strip()
        else:
            return cell.split(',')[0].strip()
    except:
        return np.NaN

In [12]:
data['country'] = data['location'].apply(get_country)

In [13]:
# Воспользуемся сервисом geonames, чтобы получить названия стран по названиям местности
def get_country_from(cell):
    if 'Greater' in cell:
        city_name = cell.split()[1].strip()
        response = requests.request("GET", f"https://www.geonames.org/search.html?q={city_name}&country=")
        country = BeautifulSoup(response.text).find_all('a')[9].text
        if country:
            return country
        else: 
            return cell
    elif 'Area' in cell:
        city_name = cell.split()[0].strip()
        response = requests.request("GET", f"https://www.geonames.org/search.html?q={city_name}&country=")
        country = BeautifulSoup(response.text).find_all('a')[9].text
        if country:
            return country
        else:
            return cell
    else:
        return cell

In [14]:
data['country'] = data['country'].apply(get_country_from)

### Получение геоточек

In [15]:
#geolocator = Nominatim(user_agent = 'your_app')

In [16]:
def get_latitude(cell):
    try:
        location = geolocator.geocode(cell)
        return location.latitude
    except:
        return 'can not find'

In [17]:
def get_longitude(cell):
    try:
        location = geolocator.geocode(cell)
        return location.longitude
    except:
        return 'can not find'

In [18]:
#data['latitude'] = data['location'].apply(get_latitude)

In [19]:
#data['longitude'] = data['location'].apply(get_longitude)

In [20]:
#data[data['latitude'] == 'can not find']['html'].count()

In [21]:
#data.query('latitude == "can not find"')

### Тип занятости

In [22]:
def get_type(cell):
    try:
        return BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__workplace-type').text.strip()
    except:
        return np.nan

In [23]:
data['type'] = data['html'].apply(get_type)
data['type'] = data['type'].fillna('unknown')

### Название компании

In [24]:
def get_company_name(cell):
    try:
        return BeautifulSoup(cell).find('a', {'class': 'ember-view t-black t-normal'}).text.strip()
    except:
        return np.nan

In [25]:
data['company'] = data['html'].apply(get_company_name)

### Сфера деятельности компании

In [26]:
def get_field(cell):
    try:
        return BeautifulSoup(cell).find('div', class_ = 'mt5 mb2').find_all('li')[1].text.strip().split('·')[1] 
    except:
        return np.nan

In [27]:
data['field'] = data['html'].apply(get_field)

### Количество сотрудников

In [28]:
def get_employees(cell):
    try:
        return BeautifulSoup(cell).find('div', class_ = 'mt5 mb2').find_all('li')[1].text.strip().split('·')[0].replace('employees', '') 
    except:
        return np.nan

In [29]:
data['employees'] = data['html'].apply(get_employees)
data['employees'] = data['employees'].str.split("·").str[0]
data['employees'] = data['employees'].str.split(" ").str[0]
data['employees'] = data['employees'].str.replace('+', '', regex=False)
data['employees'] = data['employees'].str.replace(',', '', regex=False)

In [30]:
data['employees'].unique()

array(['10001', '5001-10000', '201-500', '1001-5000', '501-1000',
       '51-200', 'See', '11-50', '1-10', 'Government', nan, 'IT',
       'Software'], dtype=object)

In [31]:
def company_size(value):
    if value == '1-10' or value == '11-50':
        return 'tiny'
    
    elif value == '51-200':
        return 'very small'
    
    elif value == '201-500':
        return 'small'
    
    elif value == '501-1000' or value == '1001-5000':
        return 'middle'
    
    elif value == '5001-10000':
        return 'large'
    
    elif value == '10001':
        return 'huge'
    
    else:
        return 'unknown'
    
data['company_size'] = data['employees'].apply(company_size)

### Дата публикации вакансии

In [32]:
def get_date(cell):
    try:
        return BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__posted-date').text.strip()
    except:
        np.nan

In [33]:
data['dt'] = data['html'].apply(get_date)

In [34]:
data['dt'].unique()

array(['13 minutes ago', '4 days ago', '6 days ago', '8 hours ago',
       '5 days ago', '2 days ago', '1 hour ago', '22 hours ago',
       '1 day ago', '5 hours ago', '10 hours ago', '4 hours ago',
       '37 minutes ago', '7 hours ago', '11 hours ago', '3 hours ago',
       '16 hours ago', '23 hours ago', '6 hours ago', '2 hours ago',
       '15 hours ago', '19 hours ago', '21 hours ago', '9 hours ago',
       '14 hours ago', '3 days ago', '13 hours ago', '18 hours ago',
       '10 minutes ago', '20 hours ago', '6 minutes ago', '4 minutes ago',
       '1 week ago', '45 minutes ago', '24 minutes ago', '28 minutes ago'],
      dtype=object)

In [35]:
#дата парсинга 
dt = datetime(2022, 9, 7, 17, 0, 0)

In [36]:
data['dt'] = data['dt'].replace({' days ago': '', ' day ago': '', '1 week ago': '7'}, regex=True)
#здесь мы убираем days и day, если написано 1 week - то заменяем на 7 - таким образом остаются только значений в днях

data['dt'] = data['dt'].str.replace('.* ago', '0', regex=True)
#все остальной - то есть hours ago/minutes ago заменяем на 0 - приняв что данные вакансии опубликованы в день парсинга

data['dt'] = data['dt'].astype(int)

data['actual_date'] = data['dt'].apply(lambda x: datetime(2022, 9, 7).date() - timedelta(days=x))
#находим разницу между датой парсинга и количество дней, когда была опубликована вакансия

data['actual_date'] = pd.to_datetime(data['actual_date'], format='%Y-%m-%d')

### Описание вакансии

In [37]:
data['description'] = data['html'].apply(lambda x: BeautifulSoup(x).find('div', {'id':'job-details'}).text.strip())

### Требуемые хард скиллы

In [38]:
skills = (['datahub', 'api', 'github', 'google analytics', 'adobe analytics', 'ibm coremetrics', 'omniture'
            'gitlab', 'erwin', 'hadoop', 'spark', 'hive'
           'databricks', 'aws', 'gcp', 'azure','excel',
            'redshift', 'bigquery', 'snowflake',  'hana'
            'grafana', 'kantar', 'spss', 
           'asana', 'basecamp', 'jira', 'dbeaver','trello', 'miro', 'salesforce', 
           'rapidminer', 'thoughtspot',  'power point',  'docker', 'jenkins','integrate.io', 'talend', 'apache nifi','aws glue','pentaho','google data flow',
             'azure data factory','xplenty','skyvia','iri voracity','xtract.io','dataddo', 'ssis',
             'hevo data','informatica','oracle data integrator','k2view','cdata sync','querysurge', 
             'rivery', 'dbconvert', 'alooma', 'stitch', 'fivetran', 'matillion','streamsets','blendo',
             'iri voracity','logstash', 'etleap', 'singer', 'apache camel','actian', 'airflow', 'luidgi', 'datastage',
           'python', 'vba', 'scala', ' r ', 'java script', 'julia', 'sql', 'matlab', 'java', 'html', 'c++', 'sas',
           'data studio', 'tableau', 'looker', 'powerbi', 'cognos', 'microstrategy', 'spotfire',
             'sap business objects','microsoft sql server', 'oracle business intelligence', 'yellowfin',
             'webfocus','sas visual analytics', 'targit', 'izenda',  'sisense', 'statsbot', 'panorama', 'inetsoft',
             'birst', 'domo', 'metabase', 'redash', 'power bi', 'alteryx', 'dataiku', 'qlik sense', 'qlikview'
          ]) 

In [39]:
def get_skills(cell):
    list_skills = []
    for skill in skills:
        if skill in cell.lower().replace('powerbi', 'power bi'):
            list_skills.append(skill)
    return ''','''.join(list_skills)

In [40]:
data['skills'] = data['description'].apply(get_skills)

In [41]:
data['actual_date'] = pd.to_datetime(data['actual_date'], format='%Y-%m-%d %H:%M:%S')

In [42]:
#промежуточный итог
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    772 non-null    int64         
 1   html          772 non-null    object        
 2   title         772 non-null    object        
 3   applicants    696 non-null    Int64         
 4   link          772 non-null    object        
 5   location      772 non-null    object        
 6   city          708 non-null    object        
 7   country       772 non-null    object        
 8   type          772 non-null    object        
 9   company       750 non-null    object        
 10  field         738 non-null    object        
 11  employees     770 non-null    object        
 12  company_size  772 non-null    object        
 13  dt            772 non-null    int64         
 14  actual_date   772 non-null    datetime64[ns]
 15  description   772 non-null    object    

## Часть 2. Преобработка данных

В этой части нужны сделать следующие действия:

- удалить дубликаты
- оставить вакансии только для аналитиков
- удаление ненужных атрибутов(столбцов)

In [43]:
final_data = data[['title', 'company', 'applicants', 'city', 'country', 'type', 'field', 'employees', 'company_size', 'actual_date', 'skills', 'location']]
            

In [44]:
final_data = final_data.replace('', np.nan)

In [45]:
final_data.duplicated().sum()

6

In [46]:
final_data.drop_duplicates(inplace=True)

In [47]:
final_data['title'][0].lower()

'stage - assistant ingénieur qualité - beyrand (h/f)'

In [48]:
#цикл, в котором мы отбираем только вакансии аналитиков 
final_data['filter'] = ''

for x in final_data.index:
    if 'data' in final_data.loc[x]['title'].lower() and 'analyst' in final_data.loc[x]['title'].lower():
        final_data['filter'].loc[x] = 'filter'
        
    elif 'bi' in final_data.loc[x]['title'].lower() and 'analyst' in final_data.loc[x]['title'].lower():
        final_data['filter'].loc[x] = 'filter'
        
    else:
        np.nan

In [49]:
final_data = final_data.query('filter == "filter"')
final_data.drop(columns='filter', inplace=True)

In [50]:
final_data['title'].unique()

array(['Online Data Analyst', 'Online Data Analyst - Belgium',
       'Data Analyst', 'Alternant/ Alternante data analyst M/F',
       'Data Analyst H/F', 'Stage - Data Analyst (H/F)',
       'Data Analyst (F/H)', 'Data Analyst (Tableau)',
       'Online Data Analyst | French Speaker',
       'Online Data Analyst | Flexible Work',
       'Online Data Analyst | REMOTE OPPORTUNITY', 'Data analyst',
       'Data Analyst (IT)', 'Data Analyst (m/f/d)',
       'Data Analyst (M/W/D)', 'Remote | Data Analyst',
       'Data Analyst SA1/SA2', 'Data analyst | Deals (M&A) | CDI | H/F',
       'Data Analyst - AdSales & StoryWorks',
       'Data Analyst with focus on solution design (m/f/d)',
       'Remote| Data Analyst', 'Data Analyst - (m/f/d)',
       'Alternance - Data Analyst/Dataviz Specialist H/F',
       'Data Analyst  - Boursorama',
       'Data Analyst in Forensic Technology Services team',
       'Data Analyst (m/w/d)', 'Analyste Exploitation Datacenter',
       'Data Analyst with Python

In [51]:
#итоговый датафрейм
final_data.head(15)

,title,company,applicants,city,country,type,field,employees,company_size,actual_date,skills,location
2,Online Data Analyst,TELUS International AI Data Solutions,12,Skara,Sweden,Remote,IT Services and IT Consulting,10001,huge,2022-09-01,NaN,"Skara, Vastra Gotaland County, Sweden"
3,Online Data Analyst - Belgium,TELUS International,11,West Flanders,Belgium,Remote,IT Services and IT Consulting,10001,huge,2022-09-01,NaN,"West Flanders, Flemish Region, Belgium"
4,Data Analyst,TELUS International AI Data Solutions,2,Mecklenburg-West Pomerania,Germany,Remote,IT Services and IT Consulting,10001,huge,2022-09-07,NaN,"Mecklenburg-West Pomerania, Germany"
5,Data Analyst,TELUS International AI Data Solutions,4,Hamburg,Germany,Remote,IT Services and IT Consulting,10001,huge,2022-09-07,NaN,"Hamburg, Germany"
6,Alternant/ Alternante data analyst M/F,STMicroelectronics,25,Rousset,France,On-site,Semiconductor Manufacturing,10001,huge,2022-09-02,"excel,python,sql,java,html,spotfire,power bi","Rousset, Provence-Alpes-Côte d'Azur, France"
11,Data Analyst,TELUS International AI Data Solutions,<NA>,Przemyśl,Przemyśl Metropolitan Area,Remote,IT Services and IT Consulting,10001,huge,2022-09-07,NaN,Przemyśl Metropolitan Area
12,Data Analyst H/F,GESER-BEST - FRANCE,7,Toulouse,France,On-site,Human Resources Services,201-500,small,2022-09-01,"excel,ssis,data studio,tableau","Toulouse, Occitanie, France"
17,Stage - Data Analyst (H/F),Liebherr Group,14,Toulouse,France,On-site,Industrial Machinery Manufacturing,10001,huge,2022-09-06,"excel,vba,sas,power bi","Toulouse, Occitanie, France"
18,Data Analyst (F/H),CELAD,2,Lille,France,On-site,IT Services and IT Consulting,1001-5000,middle,2022-09-05,"hadoop,spark,gcp,azure,python","Lille, Hauts-de-France, France"
19,Data Analyst (Tableau),Guardian Jobs,30,NaN,Gibraltar,unknown,Staffing and Recruiting,51-200,very small,2022-09-06,"api,excel,python,sql,tableau",Gibraltar


In [52]:
final_data.to_csv('/Users/denis_sl/Desktop/проект linkedin/parser_linkedin_3.csv', index = False)

In [53]:
final_data['title'].count()

279